In [ ]:
# https://pyneng.readthedocs.io/ru/latest/book/12_useful_modules/subprocess.html

Модуль subprocess позволяет создавать новые процессы. При этом он может подключаться к [стандартным потокам ввода/вывода/ошибок](http://xgu.ru/wiki/stdin) и получать код возврата.

С помощью subprocess можно, например, выполнять любые команды Linux из скрипта. И в зависимости от ситуации получать вывод или только проверять, что команда выполнилась без ошибок.

Самый простой вариант использования функции - запуск её таким образом:

In [ ]:
import subprocess

In [ ]:
result = subprocess.run('ls')

В переменной result теперь содержится специальный объект CompletedProcess. Из этого объекта можно получить информацию о выполнении процесса, например, о коде возврата:

In [ ]:
result

In [ ]:
result.returncode

Код 0 означает, что программа выполнилась успешно.

Если необходимо вызвать команду с аргументами, её нужно передавать таким образом (как список):

In [ ]:
result = subprocess.run(['ls', '-ls'])

При попытке выполнить команду с использованием wildcard-выражений, например, использовать *, возникнет ошибка:

In [ ]:
result = subprocess.run(['ls', '-ls', '*py'])

Чтобы вызывать команды, в которых используются wildcard-выражения, нужно добавлять аргумент shell и вызывать команду таким образом:

In [ ]:
result = subprocess.run('ls -ls *py', shell=True)

Ещё одна особенность функции run() - она ожидает завершения выполнения команды. Если попробовать, например, запустить команду ping, то этот аспект будет заметен:

In [ ]:
result = subprocess.run(['ping', '-c', '3', '-n', 'ya.ru'])

По умолчанию функция run возвращает результат выполнения команды на стандартный поток вывода. Если нужно получить результат выполнения команды, надо добавить аргумент stdout и указать ему значение subprocess.PIPE:

In [ ]:
result = subprocess.run(['ls', '-ls'], stdout=subprocess.PIPE)

Теперь можно получить результат выполнения команды таким образом:

In [ ]:
print(result.stdout)

Обратите внимание на букву b перед строкой. Она означает, что модуль вернул вывод в виде байтовой строки. Для перевода её в unicode есть два варианта:

- выполнить decode полученной строки
- указать аргумент encoding

Вариант с decode:

In [ ]:
print(result.stdout.decode('utf-8'))

Вариант с encoding:

In [ ]:
result = subprocess.run(['ls', '-ls'], stdout=subprocess.PIPE, encoding='utf-8')
print(result.stdout)

Иногда достаточно получения кода возврата и нужно отключить вывод результата выполнения на стандартный поток вывода, и при этом сам результат не нужен. Это можно сделать, передав функции run аргумент stdout со значением subprocess.DEVNULL:

In [ ]:
result = subprocess.run(['ls', '-ls'], stdout=subprocess.DEVNULL)
print(result.stdout)
print(result.returncode)

Если команда была выполнена с ошибкой или не отработала корректно, вывод команды попадет на стандартный поток ошибок.

Получить этот вывод можно так же, как и стандартный поток вывода:

In [ ]:
result = subprocess.run(['ping', '-c', '3', '-n', 'a'], stderr=subprocess.PIPE, encoding='utf-8')

Теперь в result.stdout пустая строка, а в result.stderr находится стандартный поток вывода:



In [ ]:
print(result.stdout)
print(result.stderr)
print(result.returncode)

Пример использования модуля subprocess (файл subprocess_run_basic.py):

In [ ]:
%%writefile subprocess_run_basic.py
import subprocess

reply = subprocess.run(['ping', '-c', '3', '-n', 'ya.ru'])

if reply.returncode == 0:
    print('Alive')
else:
    print('Unreachable')

In [ ]:
%run subprocess_run_basic.py

То есть, результат выполнения команды выводится на стандартный поток вывода.

Функция ping_ip проверяет доступность IP-адреса и возвращает True и stdout, если адрес доступен, или False и stderr, если адрес недоступен (файл subprocess_ping_function.py):

In [ ]:
%%writefile subprocess_ping_function.py
import subprocess

def ping_ip(ip_address):
    """
    Ping IP address and return tuple:
    On success:
        * True
        * command output (stdout)
    On failure:
        * False
        * error output (stderr)
    """
    reply = subprocess.run(['ping', '-c', '3', '-n', ip_address],
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           encoding='utf-8')
    if reply.returncode == 0:
        return True, reply.stdout
    else:
        return False, reply.stderr

print(ping_ip('ya.ru'))
print(ping_ip('a'))

In [ ]:
%run subprocess_ping_function.py

На основе этой функции, можно сделать функцию, которая будет проверять список IP-адресов и возвращать в результате выполнения два списка: доступные и недоступные адреса.

Если количество IP-адресов, которые нужно проверить, большое, можно использовать модуль threading или multiprocessing, чтобы ускорить проверку.

In [ ]:
import subprocess

result = subprocess.run("df -k", shell=True, stdout=subprocess.PIPE, encoding='utf-8')
print(result.stdout)
print(result.returncode)

In [ ]:
import subprocess

result = subprocess.run("du -hs $HOME", shell=True, stdout=subprocess.PIPE, encoding='utf-8')
print(result.stdout)
print(result.returncode)

In [ ]:
import subprocess

res = subprocess.Popen(['uname', '-sv'], stdout=subprocess.PIPE, encoding='utf-8')
uname = res.stdout.read().strip()
uname

In [ ]:
'Linux' in uname

In [ ]:
uname.split()

In [ ]:
import subprocess

uname = "uname"
uname_arg = "-sv"
print(f"Gathering system information with {uname} command:\n")
result = subprocess.run([uname, uname_arg], stdout=subprocess.PIPE)
print(result)